In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/self_created_python_packages')
import numpy as np
import sqlite3
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_text
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier


from database_functions import get_DataFrame_from_SQL_database, connect_to_database, write_dataframe_to_database

Mounted at /content/drive


In [ ]:
def connect_to_database(name_of_database, path = "/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/databases"):
  # Verbindung zur Datenbank herstellen
  conn = sqlite3.connect(path + "/" + name_of_database)

  # Cursor-Objekt erstellen
  cursor = conn.cursor()

  return conn, cursor

Connect to database wehre trainingsdata is saved

In [ ]:
[conn_timeseries, cur_timeseries] = connect_to_database('timeseries_database_finished.db')

Function which provides sql code to get trainingsdata

In [ ]:
def get_query_timeseries(H, window_size_rolling_window):
    return f'''
      SELECT
        da_prc.YEAR, da_prc.WEEKDAY, da_prc.MONTH, da_prc.HOUR, da_prc.DUMMY_WEEKDAY, da_prc.DA_PRC_DAY_BEFORE, da_prc.DA_AV_PRC_3_DAYS, da_prc.DA_PRC_SAME_DAY,
        da_vol.DA_VOL_DAY_BEFORE, da_vol.DA_VOL_SAME_DAY, da_vol.DA_AV_VOL_3_DAYS,
        pl_pr.PL_PR_TOTAL, pl_pr.PL_PR_GAS, pl_pr.PL_PR_WIND, pl_pr.PL_PR_LIGNITE, pl_pr.PL_PR_BC, pl_pr.PL_PR_IC, pl_pr.PL_PR_OIL, pl_pr.PL_PR_GEO, pl_pr.PL_PR_HYDRO, pl_pr.PL_PR_NAPHTA, pl_pr.PL_PR_BIO, pl_pr.PL_PR_RIVER, pl_pr.PL_PR_OTHER,
        pr_inj.INJ_SUN_{H}_h, pr_inj.INJ_ASPHAL_{H}_h, pr_inj.INJ_LNG_{H}_h, pr_inj.INJ_II_{H}_h, pr_inj.INJ_IE_{H}_h,
        pr_inj.DIFFERENZ_PR_INJ_TOTAL_{H}_h, pr_inj.DIFFERENZ_PR_INJ_GAS_{H}_h, pr_inj.DIFFERENZ_PR_INJ_WIND_{H}_h, pr_inj.DIFFERENZ_PR_INJ_LIGNITE_{H}_h, pr_inj.DIFFERENZ_PR_INJ_BC_{H}_h, pr_inj.DIFFERENZ_PR_INJ_IC_{H}_h, pr_inj.DIFFERENZ_PR_INJ_OIL_{H}_h, pr_inj.DIFFERENZ_PR_INJ_GEO_{H}_h, pr_inj.DIFFERENZ_PR_INJ_HYDRO_{H}_h, pr_inj.DIFFERENZ_PR_INJ_NAPHTA_{H}_h, pr_inj.DIFFERENZ_PR_INJ_BIO_{H}_h, pr_inj.DIFFERENZ_PR_INJ_RIVER_{H}_h, pr_inj.DIFFERENZ_PR_INJ_OTHER_{H}_h,
        pr_inj_rw.ROLLING_WINDOW_INJ_SUN_{H}_h, pr_inj_rw.ROLLING_WINDOW_INJ_ASPHAL_{H}_h, pr_inj_rw.ROLLING_WINDOW_INJ_LNG_{H}_h, pr_inj_rw.ROLLING_WINDOW_INJ_II_{H}_h, pr_inj_rw.ROLLING_WINDOW_INJ_IE_{H}_h,
        pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_TOTAL_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_GAS_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_WIND_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_LIGNITE_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_BC_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_IC_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_OIL_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_GEO_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_HYDRO_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_NAPHTA_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_BIO_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_RIVER_{H}_h, pr_inj_rw.ROLLING_WINDOW_DIFFERENZ_PR_INJ_OTHER_{H}_h,
        for_load.FORECAST_LOAD_SAME_DAY,
        for_load_rw.ROLLING_WINDOW_PAST_{window_size_rolling_window}_h_DIFFERENZ_FOR_REAL_LOAD_{H}_h,
        sys_dir.SYSTEM_DIRECTION_{H}_h, sys_dir.ROLLING_WINDOW_PAST_{window_size_rolling_window}_h_SYSTEM_DIRECTION_{H}_h,
        up_reg.UP_REG_{H}_h_0, up_reg.UP_REG_{H}_h_1, up_reg.UP_REG_{H}_h_2,
        down_reg.DOWN_REG_{H}_h_0, down_reg.DOWN_REG_{H}_h_1, down_reg.DOWN_REG_{H}_h_2,
        fi_pl.PL_FI_LOAD,
        fi_unpl.UNPL_FI_LOAD_{H}_h, fi_unpl.ROLLING_WINDOW_PAST_{window_size_rolling_window}_h_UNPL_FI_LOAD_{H}_h,
        fi_pow.POWER_FI_{H}_h, fi_pow.ROLLING_WINDOW_PAST_{window_size_rolling_window}_h_POWER_FI_{H}_h,
        wpp.DIFF_FOR_GEN_WPP_{H}_h, wpp.GENERATION_WPP_{H}_h, wpp.FORECAST_WPP,
        tr.TRADE_COUNT_{H}_h, tr.TRADE_VOLATILITY_{H}_h, tr.TRADE_VOLUME_{H}_h, tr.PRE_ID3_{H}_h, tr.AV_TRADE_COUNT_{H}_h, tr.AV_TRADE_VOLATILITY_{H}_h, tr.AV_TRADE_VOLUME_{H}_h, tr.AV_PRE_ID3_{H}_h,
        id.ID3_FILLED
      FROM
        DA_PRC_timeseries da_prc
      LEFT JOIN
        DA_VOL_timeseries da_vol
      ON
        da_prc.date = da_vol.date
      LEFT JOIN
        PLANNED_PR_timeseries pl_pr
      ON
        da_prc.date = pl_pr.date
      LEFT JOIN
        PROD_INJ_timeseries pr_inj
      ON
        da_prc.date = pr_inj.date
      LEFT JOIN
        PROD_INJ_RW_timeseries pr_inj_rw
      ON
        da_prc.date = pr_inj_rw.date
      LEFT JOIN
        FOR_LOAD_timeseries for_load
      ON
        da_prc.date = for_load.date
      LEFT JOIN
        FOR_REAL_LOAD_RW_timeseries for_load_rw
      ON
        da_prc.date = for_load_rw.date
      LEFT JOIN
        SYSTEM_DIRECTION_timeseries sys_dir
      ON
        da_prc.date = sys_dir.date
      LEFT JOIN
        UP_REG_timeseries up_reg
      ON
        da_prc.date = up_reg.date
      LEFT JOIN
        DOWN_REG_timeseries down_reg
      ON
        da_prc.date = down_reg.date
      LEFT JOIN
        FI_PL_LOAD_timeseries fi_pl
      ON
        da_prc.date = fi_pl.date
      LEFT JOIN
        FI_UNPL_LOAD_timeseries fi_unpl
      ON
        da_prc.date = fi_unpl.date
      LEFT JOIN
        FI_POWER_timeseries fi_pow
      ON
        da_prc.date = fi_pow.date
      LEFT JOIN
        WPP_timeseries wpp
      ON
        da_prc.date = wpp.date
      LEFT JOIN
        TRADING_DATA_{H}_timeseries tr
      ON
        da_prc.date = tr.date
      LEFT JOIN
        ID_PRICES_timeseries id
      ON
        da_prc.date = id.date
      '''


These functions will help us zo get training data for different training methods. They simply add filter to the sql statement which gets us the training data

In [ ]:
def get_timeseries(cursor, orig_query):
    """
    Retrieves a time series from a database using the provided cursor and query.

    Parameters:
    - cursor (database cursor): Cursor object to execute SQL queries.
    - orig_query (str): Original SQL query to fetch data.

    Returns:
    - df (pandas.DataFrame): DataFrame containing the fetched data.
    """
    cursor.execute(orig_query)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]

    df = pd.DataFrame(rows, columns=columns)

    df = df.replace([np.inf, -np.inf], 0)  # replace Inf with 0
    df = df.dropna(how='any')  # drop rows that have at least one NaN value

    return df

def get_timeseries_weekday(cursor, weekday, orig_query):
    """
    Retrieves a time series filtered by a specific weekday.

    Parameters:
    - cursor (database cursor): Cursor object to execute SQL queries.
    - weekday (int): Weekday number (0 for Monday, 1 for Tuesday, ..., 6 for Sunday).
    - orig_query (str): Original SQL query to fetch data.

    Returns:
    - df (pandas.DataFrame): DataFrame containing the fetched data filtered by weekday.
    """
    weekday_filter = f'''
        WHERE da_prc.WEEKDAY = {weekday}
    '''

    cursor.execute(orig_query + ' ' + weekday_filter)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]

    df = pd.DataFrame(rows, columns=columns)

    df = df.replace([np.inf, -np.inf], 0)  # replace Inf with 0
    df = df.dropna(how='any')  # drop rows that have at least one NaN value

    return df

def get_timeseries_hour(cursor, hour, orig_query):
    """
    Retrieves a time series filtered by a specific hour.

    Parameters:
    - cursor (database cursor): Cursor object to execute SQL queries.
    - hour (int): Hour of the day (0 to 23).
    - orig_query (str): Original SQL query to fetch data.

    Returns:
    - df (pandas.DataFrame): DataFrame containing the fetched data filtered by hour.
    """
    hour_filter = f'''
        WHERE da_prc.HOUR = {hour}
    '''
    cursor.execute(orig_query + ' ' + hour_filter)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]

    df = pd.DataFrame(rows, columns=columns)

    df = df.replace([np.inf, -np.inf], 0)  # replace Inf with 0
    df = df.dropna(how='any')  # drop rows that have at least one NaN value

    return df

def get_timeseries_weekday_and_hour(cursor, hour, weekday, orig_query):
    """
    Retrieves a time series filtered by both a specific hour and weekday.

    Parameters:
    - cursor (database cursor): Cursor object to execute SQL queries.
    - hour (int): Hour of the day (0 to 23).
    - weekday (int): Weekday number (0 for Monday, 1 for Tuesday, ..., 6 for Sunday).
    - orig_query (str): Original SQL query to fetch data.

    Returns:
    - df (pandas.DataFrame): DataFrame containing the fetched data filtered by hour and weekday.
    """
    filter = f'''
        WHERE da_prc.HOUR = {hour} AND da_prc.WEEKDAY = {weekday}
    '''

    cursor.execute(orig_query + ' ' + filter)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]

    df = pd.DataFrame(rows, columns=columns)

    df = df.replace([np.inf, -np.inf], 0)  # replace Inf with 0
    df = df.dropna(how='any')  # drop rows that have at least one NaN value

    return df


In [ ]:
def get_train_test(H, cursor, test_size, random_state, window_size_rolling_window=3, weekday=-1, hour=-1,
                  print_shape=True):
    """
    Retrieves training and test datasets based on specified parameters.

    Parameters:
    - H (str): Identifier for the data source.
    - cursor (database cursor): Cursor object to execute SQL queries.
    - test_size (float): Fraction of the data to reserve for testing.
    - random_state (int): Seed for random number generator for reproducibility.
    - window_size_rolling_window (int): Size of the rolling window for data aggregation.
    - weekday (int): Weekday filter (-1 indicates no filter).
    - hour (int): Hour filter (-1 indicates no filter).
    - print_shape (bool): Flag to print the shapes of training and test datasets.

    Returns:
    - X_train, X_test, y_train, y_test (pandas.DataFrames): Training and test data split into features and target variables.
    """
    if hour >= 0:
        if weekday >= 0:
            df = get_timeseries_weekday_and_hour(cursor, hour, weekday,
                                                 orig_query=get_query_timeseries(H, window_size_rolling_window))
        else:
            df = get_timeseries_hour(cursor, hour,
                                     orig_query=get_query_timeseries(H, window_size_rolling_window))
    else:
        if weekday >= 0:
            df = get_timeseries_weekday(cursor, weekday,
                                        orig_query=get_query_timeseries(H, window_size_rolling_window))
        else:
            df = get_timeseries(cursor, orig_query=get_query_timeseries(H, window_size_rolling_window))

    # Split the dataset into features (X) and target variables (y)
    X = df.drop('ID3_filled', axis=1)
    y = df['ID3_filled']

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    if print_shape:
        # Print the shapes of the training and test data
        print("Trainingsdaten Shape:", X_train.shape, y_train.shape)
        print("Testdaten Shape:", X_test.shape, y_test.shape)

    return X_train, X_test, y_train, y_test


def choose_right_test_size_function(test_format_string, H, cursor, test_size, random_state, window_size_rolling_window=3):
    """
    Determines the appropriate function to generate training and test datasets based on the test_format_string.

    Parameters:
    - test_format_string (str): Specifies the format or type of test data requested.
    - H (str): Identifier for the data source.
    - cursor (database cursor): Cursor object to execute SQL queries.
    - test_size (float): Fraction of the data to reserve for testing.
    - random_state (int): Seed for random number generator for reproducibility.
    - window_size_rolling_window (int): Size of the rolling window for data aggregation.

    Returns:
    - X_train, X_test, y_train, y_test (pandas.DataFrames): Training and test data split into features and target variables.
    """
    def get_wochentag(wochentag):
        """
        Converts a weekday string to its corresponding integer representation.

        Parameters:
        - wochentag (str): Weekday string ('Monday', 'Tuesday', ..., 'Sunday').

        Returns:
        - weekday_int (int): Integer representation of the weekday (0 for Monday, 1 for Tuesday, ..., 6 for Sunday).
        """
        if wochentag == 'Monday':
            weekday_int = 0
        elif wochentag == 'Tuesday':
            weekday_int = 1
        elif wochentag == 'Wednesday':
            weekday_int = 2
        elif wochentag == 'Thursday':
            weekday_int = 3
        elif wochentag == 'Friday':
            weekday_int = 4
        elif wochentag == 'Saturday':
            weekday_int = 5
        else:
            weekday_int = 6
        return weekday_int

    if len(test_format_string) <= 4:
        # Only hours format, i.e., we need hours
        first_part_of_string = test_format_string.split("_")[0]

        [X_train, X_test, y_train, y_test] = get_train_test(H, cursor, test_size, random_state,
                                                            window_size_rolling_window, hour=int(first_part_of_string),
                                                            print_shape=False)
    elif len(test_format_string) > 4 and test_format_string[-1] == 'h':
        # Weekday_hour_h format
        splitted_string = test_format_string.split("_")
        wochentag = splitted_string[0]
        stunde = splitted_string[1]

        weekday_int = get_wochentag(wochentag)

        [X_train, X_test, y_train, y_test] = get_train_test(H, cursor, test_size, random_state,
                                                            window_size_rolling_window, weekday=weekday_int,
                                                            hour=int(stunde),
                                                            print_shape=False)
    elif test_format_string == 'WHOLE_SERIES':
        # Whole series format
        [X_train, X_test, y_train, y_test] = get_train_test(H, cursor, test_size, random_state,
                                                            window_size_rolling_window, print_shape=False)
    else:
        # Single weekday format
        weekday_int = get_wochentag(test_format_string)

        [X_train, X_test, y_train, y_test] = get_train_test(H, cursor, test_size, random_state,
                                                            window_size_rolling_window, weekday=weekday_int,
                                                            print_shape=False)

    return X_train, X_test, y_train, y_test


We will iterate through different hyperparameters, exspecially the test / trainingsformat. The weekdays mean that we compute wekkday wise, the hours means hour wise and wekkday + hour means weekday + hour wise

In [ ]:
# H steht für die Anzahl der Stunden vor dem Produkt (also 3 dafür, dass wir 3 Stunden vor Produkt start den ID3 Preis schätzen wollen)
H_values = [3,2]
train_values = [0.8]

days_to_take_average_over_ = 3
window_size_rolling_window_ = 3

# das testformat sagt uns wie die test datensätze und trainingsdatensätze bestimmt werden. Für Monday wird zum beispiel nur der montag trainiert und getestet, für monday_0_h wird nur montags 0 uhr trainiert etc.
testformat = [ "Monday",	"Tuesday",	"Wednesday",	"Thursday",	"Friday",	"Saturday",	"Sunday","0_h",	"1_h",	"2_h",	"3_h",	"4_h",	"5_h",	"6_h",	"7_h",	"8_h",	"9_h",	"10_h",	"11_h",	"12_h",	"13_h",	"14_h",	"15_h",	"16_h",	"17_h",	"18_h",	"19_h",	"20_h",	"21_h",	"22_h",	"23_h",
 "Monday_0_h",	"Monday_1_h",	"Monday_2_h",	"Monday_3_h",	"Monday_4_h",	"Monday_5_h",	"Monday_6_h",	"Monday_7_h",	"Monday_8_h",	"Monday_9_h",	"Monday_10_h",	"Monday_11_h",	"Monday_12_h",	"Monday_13_h",	"Monday_14_h",	"Monday_15_h",	"Monday_16_h",	"Monday_17_h",
"Monday_18_h",	"Monday_19_h",	"Monday_20_h",	"Monday_21_h",	"Monday_22_h",	"Monday_23_h",
"Tuesday_0_h",	"Tuesday_1_h",	"Tuesday_2_h",	"Tuesday_3_h",	"Tuesday_4_h",	"Tuesday_5_h",	"Tuesday_6_h",	"Tuesday_7_h",	"Tuesday_8_h",	"Tuesday_9_h",	"Tuesday_10_h",	"Tuesday_11_h",	"Tuesday_12_h",	"Tuesday_13_h",	"Tuesday_14_h",	"Tuesday_15_h",	"Tuesday_16_h",	"Tuesday_17_h",	"Tuesday_18_h",	"Tuesday_19_h",	"Tuesday_20_h",	"Tuesday_21_h",	"Tuesday_22_h",	"Tuesday_23_h",
"Wednesday_0_h",	"Wednesday_1_h",	"Wednesday_2_h",	"Wednesday_3_h",	"Wednesday_4_h",	"Wednesday_5_h",	"Wednesday_6_h",	"Wednesday_7_h",	"Wednesday_8_h",	"Wednesday_9_h",	"Wednesday_10_h",	"Wednesday_11_h",	"Wednesday_12_h",	"Wednesday_13_h",	"Wednesday_14_h",	"Wednesday_15_h",	"Wednesday_16_h",	"Wednesday_17_h",
"Wednesday_18_h",	"Wednesday_19_h",	"Wednesday_20_h",	"Wednesday_21_h",	"Wednesday_22_h",	"Wednesday_23_h",
"Thursday_0_h",	"Thursday_1_h",	"Thursday_2_h",	"Thursday_3_h",	"Thursday_4_h",	"Thursday_5_h",	"Thursday_6_h",	"Thursday_7_h",	"Thursday_8_h",	"Thursday_9_h",	"Thursday_10_h",	"Thursday_11_h",	"Thursday_12_h",	"Thursday_13_h",	"Thursday_14_h",	"Thursday_15_h",	"Thursday_16_h",	"Thursday_17_h",	"Thursday_18_h",	"Thursday_19_h",	"Thursday_20_h",	"Thursday_21_h",	"Thursday_22_h",	"Thursday_23_h",
"Friday_0_h",	"Friday_1_h",	"Friday_2_h",	"Friday_3_h",	"Friday_4_h",	"Friday_5_h",	"Friday_6_h",	"Friday_7_h",	"Friday_8_h",	"Friday_9_h",	"Friday_10_h",	"Friday_11_h",	"Friday_12_h",	"Friday_13_h",	"Friday_14_h",	"Friday_15_h",	"Friday_16_h",	"Friday_17_h",	"Friday_18_h",	"Friday_19_h",	"Friday_20_h",	"Friday_21_h",	"Friday_22_h",	"Friday_23_h",
"Saturday_0_h",	"Saturday_1_h",	"Saturday_2_h",	"Saturday_3_h",	"Saturday_4_h",	"Saturday_5_h",	"Saturday_6_h",	"Saturday_7_h",	"Saturday_8_h",	"Saturday_9_h",	"Saturday_10_h",	"Saturday_11_h",	"Saturday_12_h",	"Saturday_13_h",	"Saturday_14_h",	"Saturday_15_h",	"Saturday_16_h",	"Saturday_17_h",	"Saturday_18_h",	"Saturday_19_h",	"Saturday_20_h",	"Saturday_21_h",	"Saturday_22_h",	"Saturday_23_h",
"Sunday_0_h",	"Sunday_1_h",	"Sunday_2_h",	"Sunday_3_h",	"Sunday_4_h",	"Sunday_5_h",	"Sunday_6_h",	"Sunday_7_h",	"Sunday_8_h",	"Sunday_9_h",	"Sunday_10_h",	"Sunday_11_h",	"Sunday_12_h",	"Sunday_13_h",	"Sunday_14_h",	"Sunday_15_h",	"Sunday_16_h",	"Sunday_17_h",	"Sunday_18_h",	"Sunday_19_h",	"Sunday_20_h",	"Sunday_21_h",	"Sunday_22_h",	"Sunday_23_h",
              "WHOLE_SERIES"
]


## Random Forest Regression

In [ ]:

# Define the hyperparameters to be tested
hyperparameters_rf = {
    'n_estimators': [1, 10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def random_forest_regression(cur_timeseries, hyperparameters, H_values, train_values, test_format_string,random_state,  threshold = 0.0001, ordner_link = '\\content\\drive\\MyDrive\\01_Data_AI_SS24\\Project_Turkish_Energy_Market\\DecisionTreeRules\\RandomForest\\Regression'):

  """
  test_format_string = 'whole_timeseries', 'Monday', 'Tuesday', .., 'Friday', '0_h', '1_h', .., '23_h', 'Monday_0_h', .., 'Monday_23_h', ..., 'Friday_23_h'

  """

  # Initialize a DataFrame to store the results
  results = pd.DataFrame(columns=['H', 'test_size', 'n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'MAE', 'MAPE', 'MSE'])

  [conn_rf_regr_, cur_rf_regr_] = connect_to_database(f'RF_Regression_test_format_string_{test_format_string}.db', path = ordner_link)

  for H in H_values:

    print('H: ' +str(H))

    table_name_feature = f'Table_H_{2}_test_size_2_n_estimators_100_max_depth_name_30_min_samples_split_10_min_samples_leaf_4_feature_importance'

    # Abfrage, um zu überprüfen, ob die Tabelle existiert
    cur_rf_regr_.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name_feature,))
    table_exists_feat = cur_rf_regr_.fetchone()

    # if table_exists_feat:
    #   continue

    test_size = round(1 - 0.8,1)
    # Den Float in eine Zeichenkette umwandeln
    float_str = str(test_size)

    # Die letzte Ziffer nach dem Dezimalpunkt extrahieren
    last_digit = int(float_str.split('.')[-1][-1])

    # Name der zu überprüfenden Tabelle


    [X_train, X_test, y_train, y_test] = choose_right_test_size_function(test_format_string, H, cur_timeseries, test_size, random_state)

    # Perform an initial feature importance analysis with default hyperparameters
    initial_rf = RandomForestRegressor(random_state=random_state)
    initial_rf.fit(X_train, y_train)
    initial_importances = initial_rf.feature_importances_

    # Set an absolute threshold for feature importance
    threshold = 0.0001
    important_indices = np.where(initial_importances > threshold)[0]
    important_features = X_train.columns[important_indices]

    # Reduce X_train and X_test to the important features
    X_train_reduced = X_train[important_features]
    X_test_reduced = X_test[important_features]

    # Loop through the hyperparameters and train the model
    for n_estimators in hyperparameters['n_estimators']:
        for max_depth in hyperparameters['max_depth']:
            for min_samples_split in hyperparameters['min_samples_split']:
                for min_samples_leaf in hyperparameters['min_samples_leaf']:

                    if max_depth is None:
                      max_depth_name = "None"
                    else:
                      max_depth_name = max_depth

                    # Name der zu überprüfenden Tabelle
                    table_name_feature = f'Table_H_{H}_test_size_{str(last_digit)}_n_estimators_{str(n_estimators)}_max_depth_name_{str(max_depth_name)}_min_samples_split_{str(min_samples_split)}_min_samples_leaf_{str(min_samples_leaf)}_feature_importance'

                    # Abfrage, um zu überprüfen, ob die Tabelle existiert
                    cur_rf_regr_.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name_feature,))
                    table_exists_feat = cur_rf_regr_.fetchone()

                    if table_exists_feat:
                        print(f"Tabelle '{table_name_feature}' existiert.")
                        continue

                    table_name = f'Table_H_{H}_test_size_{str(last_digit)}_n_estimators_{str(n_estimators)}_max_depth_name_{str(max_depth_name)}_min_samples_split_{str(min_samples_split)}_min_samples_leaf_{str(min_samples_leaf)}_ypred'

                    # Abfrage, um zu überprüfen, ob die Tabelle existiert
                    cur_rf_regr_.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
                    table_exists = cur_rf_regr_.fetchone()

                    if table_exists:
                        print(f"Tabelle '{table_name}' existiert.")
                        continue

                    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, random_state=random_state)
                    rf.fit(X_train_reduced, y_train)

                    # Die Merkmalsbedeutungen extrahieren
                    importances = rf.feature_importances_

                    # Sortieren der Merkmale nach ihrer Bedeutung
                    indices = np.argsort(importances)[::-1]

                    # Feature-Namen (in diesem Fall die Spaltennamen von X_train)
                    feature_names = X_train_reduced.columns

                    # Erstellen eines DataFrames mit Feature-Namen und ihren Bedeutungen
                    feature_importance_df = pd.DataFrame({
                        'Feature': feature_names[indices],
                        'Importance': importances[indices]
                    })


                    # Make predictions
                    y_pred = rf.predict(X_test_reduced)

                    all_data = X_test.copy()
                    all_data['real_ID3'] = y_test
                    all_data['pred_ID3'] = y_pred

                    write_dataframe_to_database(feature_importance_df, table_name_feature, conn_rf_regr_, cur_rf_regr_, show_succ_msg = False)

                    write_dataframe_to_database(all_data, table_name, conn_rf_regr_, cur_rf_regr_, show_succ_msg = False)

                    # Calculate the metrics
                    mae = mean_absolute_error(y_test, y_pred)
                    mape = mean_absolute_percentage_error(y_test, y_pred)
                    mse = mean_squared_error(y_test, y_pred)

                    # Store the results
                    new_row = {'H':H, 'test_size':test_size,'n_estimators': n_estimators, 'max_depth': max_depth_name, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'MAE': mae, 'MAPE': mape, 'MSE': mse}
                    new_row_df = pd.DataFrame([new_row])
                    results = pd.concat([results, new_row_df], ignore_index=True)

                    #achtung hier werden duplikate eingefügt in den dataframe hinterher, das heißt der wird zziemlich groß, danach muss ich duplikate löschen aus dem sql tabelle
                    write_dataframe_to_database(new_row_df, f'results_error', conn_rf_regr_, cur_rf_regr_, show_succ_msg = False)

  conn_rf_regr_.close()

  return results

In [ ]:
# dieser code abschnitt iteriert jetzt zwischen den versschiedenen hyperparametern, testgrößen, Stunden vor vorhersage und test formaten durch, die Ergebnisse werden in eine Datenbank geschrieben
# diese datenbank ist unter '/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/DecisionTreeRules/RandomForest/Regression' zu finden, genau so wie die einzelnnen entscheidungsregeln


for i in range(len(testformat)):
  test_format_string_ = testformat[i]
  print(test_format_string_)

  random_state_ = i
  results_rf_ = random_forest_regression(cur_timeseries, hyperparameters_rf, H_values, train_values, test_format_string_, random_state_)





Sunday_0_h


OperationalError: unable to open database file

## Boosted Regression

In [ ]:
hyperparameters_bt = {
    'n_estimators': [200], #[10, 50, 100],
    'learning_rate': [0.1, 0.01, 0.05],
    'max_depth': [None, 1, 3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def boosted_regression(cur_timeseries, hyperparameters, H_values, train_values, test_format_string,random_state, ordner_link = '/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/DecisionTreeRules/BoostedTree/Regression'):
  # Initialize a DataFrame to store the results
  results = pd.DataFrame(columns=['H', 'test_size','n_estimators', 'learning_rate', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'MAE', 'MAPE', 'MSE'])

  [conn_bt_regr_, cur_bt_regr_] = connect_to_database(f'BT_Regression_test_format_string_{test_format_string}.db', path = ordner_link)

  for H in H_values:
    print('H: ' +str(H))

    test_size = round(1 - 0.8,1)
    # Den Float in eine Zeichenkette umwandeln
    float_str = str(test_size)

    # Die letzte Ziffer nach dem Dezimalpunkt extrahieren
    last_digit = int(float_str.split('.')[-1][-1])

    # Name der zu überprüfenden Tabelle
    [X_train, X_test, y_train, y_test] = choose_right_test_size_function(test_format_string, H, cur_timeseries, test_size, random_state)

    # Perform an initial feature importance analysis with default hyperparameters
    initial_bt = GradientBoostingRegressor(random_state=random_state)

    initial_bt.fit(X_train, y_train)
    initial_importances = initial_bt.feature_importances_

    # Set an absolute threshold for feature importance
    threshold = 0.0001
    important_indices = np.where(initial_importances > threshold)[0]
    important_features = X_train.columns[important_indices]

    # Reduce X_train and X_test to the important features
    X_train_reduced = X_train[important_features]
    X_test_reduced = X_test[important_features]

    [X_train, X_test, y_train, y_test] = choose_right_test_size_function(test_format_string, H, cur_timeseries, test_size, random_state)
    # Loop through the hyperparameters and train the model
    for n_estimators in hyperparameters['n_estimators']:
        for learning_rate in hyperparameters['learning_rate']:
          if learning_rate == 0.1:
            lr_str = "1"
          elif learning_rate == 0.01:
            lr_str = "01"
          elif learning_rate == 0.05:
            lr_str = "05"
            for max_depth in hyperparameters['max_depth']:
                for min_samples_split in hyperparameters['min_samples_split']:
                    for min_samples_leaf in hyperparameters['min_samples_leaf']:
                        if max_depth is None:
                          max_depth_name = "None"
                        else:
                          max_depth_name = max_depth

                        # Name der zu überprüfenden Tabelle
                        table_name_feature = f'Table_H_{H}_test_size_{str(last_digit)}_n_estimators_{str(n_estimators)}_learning_rate_{lr_str}_max_depth_name_{str(max_depth_name)}_min_samples_split_{str(min_samples_split)}_min_samples_leaf_{str(min_samples_leaf)}_feature_importance'

                        # Abfrage, um zu überprüfen, ob die Tabelle existiert
                        cur_bt_regr_.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name_feature,))
                        table_exists_feat = cur_bt_regr_.fetchone()

                        if table_exists_feat:
                            print(f"Tabelle '{table_name_feature}' existiert.")
                            continue

                        table_name = f'Table_H_{H}_test_size_{str(last_digit)}_n_estimators_{str(n_estimators)}_learning_rate_{lr_str}_max_depth_name_{str(max_depth_name)}_min_samples_split_{str(min_samples_split)}_min_samples_leaf_{str(min_samples_leaf)}_ypred'

                        # Abfrage, um zu überprüfen, ob die Tabelle existiert
                        cur_bt_regr_.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
                        table_exists = cur_bt_regr_.fetchone()

                        if table_exists:
                            print(f"Tabelle '{table_name}' existiert.")
                            continue

                        gbr = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, random_state=random_state)
                        gbr.fit(X_train_reduced, y_train)

                        # Die Merkmalsbedeutungen extrahieren
                        importances = gbr.feature_importances_

                        # Sortieren der Merkmale nach ihrer Bedeutung
                        indices = np.argsort(importances)[::-1]

                        # Feature-Namen (in diesem Fall die Spaltennamen von X_train)
                        feature_names = X_train_reduced.columns

                        # Erstellen eines DataFrames mit Feature-Namen und ihren Bedeutungen
                        feature_importance_df = pd.DataFrame({
                            'Feature': feature_names[indices],
                            'Importance': importances[indices]
                        })

                        # Make predictions
                        y_pred = gbr.predict(X_test_reduced)

                        all_data = X_test.copy()
                        all_data['real_ID3'] = y_test
                        all_data['pred_ID3'] = y_pred

                        if max_depth is None:
                          max_depth_name = "None"
                        else:
                          max_depth_name = max_depth

                        write_dataframe_to_database(all_data, f'Table_H_{H}_test_size_{last_digit}_learning_rate_{lr_str}_n_estimators_{str(n_estimators)}_max_depth_name_{str(max_depth_name)}_min_samples_split_{str(min_samples_split)}_min_samples_leaf_{str(min_samples_leaf)}_ypred', conn_bt_regr_, cur_bt_regr_, show_succ_msg = False)

                        # Calculate the metrics
                        mae = mean_absolute_error(y_test, y_pred)
                        mape = mean_absolute_percentage_error(y_test, y_pred)
                        mse = mean_squared_error(y_test, y_pred)

                        new_row = {'H':H, 'test_size':test_size,'n_estimators': n_estimators, 'max_depth': max_depth_name, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'MAE': mae, 'MAPE': mape, 'MSE': mse}
                        new_row_df = pd.DataFrame([new_row])
                        results = pd.concat([results, new_row_df], ignore_index=True)

                        write_dataframe_to_database(new_row_df, f'results_error', conn_bt_regr_, cur_bt_regr_, show_succ_msg = False)


  conn_bt_regr_.close()
  return results

In [ ]:
# dieser code abschnitt iteriert jetzt zwischen den versschiedenen hyperparametern, testgrößen, Stunden vor vorhersage und test formaten durch, die Ergebnisse werden in eine Datenbank geschrieben
# diese datenbank ist unter '/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/DecisionTreeRules/BoostedTree/Regression' zu finden, genau so wie die einzelnnen entscheidungsregeln

for i in range(len(testformat)):
  test_format_string_ = testformat[i]
  print(test_format_string_)
  random_state_ = i

  results_bt_ = boosted_regression(cur_timeseries, hyperparameters_bt, H_values, train_values, test_format_string_, random_state_)

Monday
H: 3
H: 2
Tuesday
H: 3
H: 2
Wednesday
H: 3
H: 2
Thursday
H: 3
H: 2
Friday
H: 3
H: 2
Saturday
H: 3
H: 2
Sunday
H: 3
H: 2
0_h
H: 3
H: 2
1_h
H: 3
H: 2
2_h
H: 3
H: 2
3_h
H: 3
H: 2
4_h
H: 3
H: 2
5_h
H: 3
H: 2
6_h
H: 3
H: 2
7_h
H: 3
H: 2


## Classification

### Bin creating with percentage

The idea is the following: either you have the possibility to say, that your price borders should contain always 5 % of the adta. Or you say, that you want price borders with an interval length of f.e. 5, then, if the model gets the correct class, it has an MAE of max. 10. Due to time limits, the second idea is only partially finished.

In [ ]:
def label_prices_by_quantiles(df, quantile):

    if (1/quantile) % 1 != 0 or quantile >= 1:
      print('Bitte wähle Zahl, sodass diese aufsummiert 1 ergibt')
      return df
    quantiles = np.arange(quantile, 1+  quantile, quantile) * 100

    df = pd.DataFrame(df)
    """
    Labels prices in a DataFrame based on quantiles.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the prices.
    quantiles (list): A list of quantiles to use for labeling.

    Returns:
    pandas.DataFrame: A new DataFrame with the same columns as the input DataFrame,
                      but with an additional column 'price_label' containing the
                      labels for each price.
    """
    # Calculate the quantiles
    q = np.percentile(df['ID3_filled'], quantiles)


    # Create a new column for the labels
    df['Label_ID3'] = 0

    # Label the prices based on the quantiles
    for i, q_val in enumerate(q):
        if i == 0:
            df.loc[df['ID3_filled'] < q_val, 'Label_ID3'] = i + 1
        else:
            df.loc[(df['ID3_filled'] < q_val) & (df['ID3_filled'] > q[i-1]), 'Label_ID3'] = i + 1


    # Return the DataFrame with the labels
    return df, q

def label_prices_by_interval(df, interval):

    df = pd.DataFrame(df)
    """
    Labels prices in a DataFrame based on quantiles.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the prices.
    interval (value): A value to use for labeling.

    Returns:
    pandas.DataFrame: A new DataFrame with the same columns as the input DataFrame,
                      but with an additional column 'price_label' containing the
                      labels for each price.
    """

    # Calculate the minimum and maximum values
    min_val = df['ID3_filled'].min()
    max_val = df['ID3_filled'].max()
    # Create a new column for the labels
    df['Label_ID3'] = 0

    interval_values = [min_val + i * interval for i in range(int((max_val - min_val) / interval) + 1) ]

    # Label the prices based on the interval
    for i in range(int((max_val - min_val) / interval) + 1):
        df.loc[(df['ID3_filled'] >= min_val + i * interval) & (df['ID3_filled'] < min_val + (i + 1) * interval), 'Label_ID3'] = i + 1

    # Return the DataFrame with the labels
    return df, interval_values

In [ ]:
def get_train_test_classifier_quantile(quantile, H, cursor, test_size, random_state, window_size_rolling_window = 3, weekday = -1, hour = -1,
                  print_shape = True):
    if hour >= 0:
        if weekday >= 0:
            df = get_timeseries_weekday_and_hour(cursor, hour, weekday,
                                                 orig_query = get_query_timeseries(H, window_size_rolling_window))
        else:
            df = get_timeseries_hour(cursor, hour,
                                    orig_query = get_query_timeseries(H, window_size_rolling_window))
    else:
        if weekday >= 0:
            df = get_timeseries_weekday_and_hour(cursor, weekday,
                                                 orig_query = get_query_timeseries(H, window_size_rolling_window))
        else:
            df = get_timeseries(cursor, orig_query = get_query_timeseries(H, window_size_rolling_window))


    # Teilen Sie den Datensatz in Features (X) und Zielvariablen (y) auf
    X = df.drop('ID3_filled', axis=1)
    y = pd.DataFrame(df['ID3_filled'])

    [y, q] = label_prices_by_quantiles(y, quantile)

    # Teilen Sie die Daten in Trainings- und Testdaten auf
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    if print_shape:
        # Überprüfen Sie die Form der Trainings- und Testdaten
        print("Trainingsdaten Shape:", X_train.shape, y_train.shape)
        print("Testdaten Shape:", X_test.shape, y_test.shape)

    return X_train, X_test, y_train, y_test, q



def choose_right_test_size_function_classifier_quantile(quantile, test_format_string, H, cursor, test_size, random_state, window_size_rolling_window = 3):

  def get_wochentag(wochentag):
    if wochentag == 'Monday':
      weekday_int = 0
    elif wochentag == 'Tuesday':
      weekday_int = 1
    elif wochentag == 'Wednesday':
      weekday_int = 2

    elif wochentag == 'Thursday':
      weekday_int = 3

    elif wochentag == 'Friday':
      weekday_int = 4

    elif wochentag == 'Saturday':
      weekday_int = 5

    else:
      weekday_int = 6
    return weekday_int


  if len(test_format_string) <= 4:
    # das kann nur stundenformat sein, d.h. wir brauchen stunden
    first_part_of_string = test_format_string.split("_")[0]

    [X_train, X_test, y_train, y_test, q]  = get_train_test_classifier_quantile(quantile, H, cursor, test_size, random_state, window_size_rolling_window, hour = int(first_part_of_string),
                  print_shape = False)
  elif len(test_format_string) > 4 and test_format_string[-1] == 'h':
    # das ist jetzt wochentag_stunde_h
    splitted_string = test_format_string.split("_")
    wochentag = splitted_string[0]
    stunde = splitted_string[1]

    weekday_int = get_wochentag(wochentag)

    [X_train, X_test, y_train, y_test, q]  = get_train_test_classifier_quantile(quantile, H, cursor, test_size,random_state, window_size_rolling_window,weekday= weekday_int, hour = int(stunde),
                  print_shape = False)
  elif test_format_string == 'WHOLE_SERIES':
    [X_train, X_test, y_train, y_test, q]  = get_train_test_classifier_quantile(quantile, H, cursor, test_size, random_state,window_size_rolling_window, print_shape = False)

  else:
    # das sind jetzt einzelne wochentage
    weekday_int = get_wochentag(test_format_string)

    [X_train, X_test, y_train, y_test, q]  = get_train_test_classifier_quantile(quantile, H, cursor, test_size, random_state, window_size_rolling_window, print_shape = False)

  y_train = y_train['Label_ID3'].ravel()
  y_test =  y_test['Label_ID3'].ravel()

  return X_train, X_test, y_train, y_test, q



# interval

def get_train_test_classifier_interval(interval, H, cursor, test_size, random_state, window_size_rolling_window = 3, weekday = -1, hour = -1,
                  print_shape = True):
    if hour >= 0:
        if weekday >= 0:
            df = get_timeseries_weekday_and_hour(cursor, hour, weekday,
                                                 orig_query = get_query_timeseries(H, window_size_rolling_window))
        else:
            df = get_timeseries_hour(cursor, hour,
                                    orig_query = get_query_timeseries(H, window_size_rolling_window))
    else:
        if weekday >= 0:
            df = get_timeseries_weekday(cursor, weekday,
                                                 orig_query = get_query_timeseries(H, window_size_rolling_window))
        else:
            df = get_timeseries(cursor, H, window_size_rolling_window)


    # Teilen Sie den Datensatz in Features (X) und Zielvariablen (y) auf
    X = df.drop('ID3_filled', axis=1)
    y = pd.DataFrame(df['ID3_filled'])

    [y, q] = label_prices_by_quantiles(y, interval)

    # Teilen Sie die Daten in Trainings- und Testdaten auf
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    if print_shape:
        # Überprüfen Sie die Form der Trainings- und Testdaten
        print("Trainingsdaten Shape:", X_train.shape, y_train.shape)
        print("Testdaten Shape:", X_test.shape, y_test.shape)

    return X_train, X_test, y_train, y_test, q



### Fitting

Random Forest

In [ ]:
# Define the hyperparameters to be tested
hyperparameters_rf_clf = {
    'n_estimators': [1, 10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def random_forest_classification(quantile_interval_values, cur_timeseries, hyperparameters, H_values, train_values, test_format_string,random_state, interval = False,
                                 ordner_link = '/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/DecisionTreeRules/RandomForest/Classification'):

  """
  testformat = 'whole_timeseries', 'Monday', 'Tuesday',.., 'Friday', '0_h', '1_h',.., '23_h', 'Monday_0_h',.., 'Monday_23_h',..., 'Friday_23_h'

  """

  # Initialize a DataFrame to store the results
  results = pd.DataFrame(columns=['H', 'test_size', 'n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'accuracy'])

  [conn_rf_clf_, cur_rf_clf_] = connect_to_database(f'RF_Classification_testformat_{test_format_string}.db', path = ordner_link)

  for quantile in quantile_interval_values:
    for H in H_values:
      if interval:
        quantile_interval_name = 'Interval'
      else:
        quantile_interval_name = 'Quantile'

      print('H: ' +str(H))

      table_name_feature = f'Table_H_{2}_test_size_2_n_estimators_100_max_depth_name_30_min_samples_split_10_min_samples_leaf_4_ypred'

      # Abfrage, um zu überprüfen, ob die Tabelle existiert
      cur_rf_clf_.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name_feature,))
      table_exists_feat = cur_rf_clf_.fetchone()

      if table_exists_feat:
        continue

      test_size = round(1 - 0.8,1)
      # Den Float in eine Zeichenkette umwandeln
      float_str = str(test_size)

      # Die letzte Ziffer nach dem Dezimalpunkt extrahieren
      last_digit = int(float_str.split('.')[-1][-1])

      # Name der zu überprüfenden Tabelle
      [X_train, X_test, y_train, y_test, q] = choose_right_test_size_function_classifier_quantile(quantile, test_format_string, H, cur_timeseries, test_size, random_state)

      # Perform an initial feature importance analysis with default hyperparameters
      initial_rf = RandomForestClassifier(random_state=random_state)
      initial_rf.fit(X_train, y_train)
      initial_importances = initial_rf.feature_importances_

      # Set an absolute threshold for feature importance
      threshold = 0.0001
      important_indices = np.where(initial_importances > threshold)[0]
      important_features = X_train.columns[important_indices]

      # Reduce X_train and X_test to the important features
      X_train_reduced = X_train[important_features]
      X_test_reduced = X_test[important_features]



      if interval:
        [X_train, X_test, y_train, y_test, q] = choose_right_test_size_function_classifier_interval(quantile, test_format_string, H, cur_timeseries, test_size, random_state)
        q_df = pd.DataFrame(q, columns = ['Interval'])
        write_dataframe_to_database(q_df, f'interval_vector_H_{H}_test_size_{last_digit}', conn_rf_clf_, cur_rf_clf_, show_succ_msg = False)
      else:
        [X_train, X_test, y_train, y_test, q] = choose_right_test_size_function_classifier_quantile(quantile, test_format_string, H, cur_timeseries, test_size, random_state)
        q_df = pd.DataFrame(q, columns = ['Quantil'])
        write_dataframe_to_database(q_df, f'quantile_vector_H_{H}_test_size_{last_digit}', conn_rf_clf_, cur_rf_clf_, show_succ_msg = False)

      # Loop through the hyperparameters and train the model
      for n_estimators in hyperparameters['n_estimators']:
          for max_depth in hyperparameters['max_depth']:
              if max_depth is None:
                max_depth_name = "None"
              else:
                max_depth_name = max_depth
              for min_samples_split in hyperparameters['min_samples_split']:
                  for min_samples_leaf in hyperparameters['min_samples_leaf']:

                      table_name = f'Table_{quantile_interval_name}_H_{H}_test_size_{str(last_digit)}_n_estimators_{str(n_estimators)}_max_depth_name_{str(max_depth_name)}_min_samples_split_{str(min_samples_split)}_min_samples_leaf_{str(min_samples_leaf)}_feature_importance'

                      # Abfrage, um zu überprüfen, ob die Tabelle existiert
                      cur_rf_clf_.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
                      table_exists = cur_rf_clf_.fetchone()

                      if table_exists:
                          print(f"Tabelle '{table_name}' existiert.")
                          continue

                      rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, random_state=random_state)
                      rf.fit(X_train_reduced, y_train)

                      # Die Merkmalsbedeutungen extrahieren
                      importances = rf.feature_importances_

                      # Sortieren der Merkmale nach ihrer Bedeutung
                      indices = np.argsort(importances)[::-1]

                      # Feature-Namen (in diesem Fall die Spaltennamen von X_train)
                      feature_names = X_train.columns

                      # Erstellen eines DataFrames mit Feature-Namen und ihren Bedeutungen
                      feature_importance_df = pd.DataFrame({
                          'Feature': feature_names[indices],
                          'Importance': importances[indices]
                      })


                      write_dataframe_to_database(feature_importance_df,
                                                  f'Table_{quantile_interval_name}_H_{H}_test_size_{str(last_digit)}_n_estimators_{str(n_estimators)}_max_depth_name_{str(max_depth_name)}_min_samples_split_{str(min_samples_split)}_min_samples_leaf_{str(min_samples_leaf)}_feature_importance',
                                                  conn_rf_clf_, cur_rf_clf_, show_succ_msg = False)


                      # Save the decision tree's rules to a text file
                      # tree_rules = export_text(rf.estimators_[0], feature_names=X_train.columns.tolist())
                      # with open(ordner_link +"/"+f'decision_tree_rules_rf_{n_estimators}_{max_depth}_{min_samples_split}_{min_samples_leaf}_{H}_{test_size}_{testformat}.txt', 'w') as f:
                      #     f.write(tree_rules)


                      # Make predictions
                      y_pred = rf.predict(X_test_reduced)



                      all_data = X_test.copy()
                      all_data['ID3_labels'] = y_test
                      all_data['pred_ID3_labels'] = y_pred

                      if max_depth is None:
                        max_depth_name = "None"
                      else:
                        max_depth_name = max_depth

                      write_dataframe_to_database(all_data, f'Table_H_{H}_test_size_{str(last_digit)}_n_estimators_{str(n_estimators)}_max_depth_name_{str(max_depth_name)}_min_samples_split_{str(min_samples_split)}_min_samples_leaf_{str(min_samples_leaf)}_ypred', conn_rf_clf_, cur_rf_clf_, show_succ_msg = False)

                      # Calculate the metrics
                      accuracy = accuracy_score(y_test, y_pred)

                      #print(accuracy)
                      #classification_report_ = classification_report(y_test, y_pred)

                     # print(classification_report_)
                      #confusion_matrix_ = confusion_matrix(y_test, y_pred)

                      # Store the results
                      new_row = {'H':H, 'test_size':test_size,'n_estimators': n_estimators, 'max_depth': max_depth_name, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'accuracy': accuracy} #, 'classification_report': classification_report_, 'confusion_matrix': confusion_matrix_}
                      new_row_df = pd.DataFrame([new_row])

                      results = pd.concat([results, new_row_df], ignore_index=True)

                      write_dataframe_to_database(new_row_df, f'{quantile_interval_name}_results_error', conn_rf_clf_, cur_rf_clf_, show_succ_msg = False)

  conn_rf_clf_.close()

  return results

In [ ]:
# dieser code abschnitt iteriert jetzt zwischen den versschiedenen hyperparametern, testgrößen, Stunden vor vorhersage und test formaten durch, die Ergebnisse werden in eine Datenbank geschrieben
# diese datenbank ist unter '/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/DecisionTreeRules/NN/Regression' zu finden, genau so wie die einzelnnen entscheidungsregeln

for i in range(len(testformat)):
  test_format_string_ = testformat[i]
  random_state_ = i
  print(test_format_string_)

  results_rf_clf_ = random_forest_classification([0.05], cur_timeseries, hyperparameters_rf_clf, H_values, train_values, test_format_string_,random_state_)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Tabelle 'Table_Quantile_H_3_test_size_2_n_estimators_100_max_depth_name_20_min_samples_split_5_min_samples_leaf_2_feature_importance' existiert.
Tabelle 'Table_Quantile_H_3_test_size_2_n_estimators_100_max_depth_name_20_min_samples_split_5_min_samples_leaf_4_feature_importance' existiert.
Tabelle 'Table_Quantile_H_3_test_size_2_n_estimators_100_max_depth_name_20_min_samples_split_10_min_samples_leaf_1_feature_importance' existiert.
Tabelle 'Table_Quantile_H_3_test_size_2_n_estimators_100_max_depth_name_20_min_samples_split_10_min_samples_leaf_2_feature_importance' existiert.
Tabelle 'Table_Quantile_H_3_test_size_2_n_estimators_100_max_depth_name_20_min_samples_split_10_min_samples_leaf_4_feature_importance' existiert.
Tabelle 'Table_Quantile_H_3_test_size_2_n_estimators_100_max_depth_name_30_min_samples_split_2_min_samples_leaf_1_feature_importance' existiert.
Tabelle 'Table_Quantile_H_3_test_size_2_n_estimators_100_max

Boosted Classification

In [ ]:
hyperparameters_bg = {
    'n_estimators': [10, 50, 100, 200],
    'learning_rate': [0.1, 0.01, 0.05],
    'max_depth': [1, 3, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

def boosted_classification(X_train, X_test, y_train, y_test, hyperparameters, H, test_size, testformat,random_state, ordner_link = '/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/DecisionTreeRules/BoostedTree/Classification'):
  # Initialize a DataFrame to store the results
  results = pd.DataFrame(columns=['n_estimators', 'learning_rate', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'accuracy', 'classification_report', 'confusion_matrix'])

  [conn_bt_clf_, cur_bt_clf_] = connect_to_database(f'BT_Classification_{H}_h_test_size_{str(test_size)}_testformat_{testformat}.db', path = ordner_link)

  # Loop through the hyperparameters and train the model
  for n_estimators in hyperparameters['n_estimators']:
      print(n_estimators)
      for learning_rate in hyperparameters['learning_rate']:
          for max_depth in hyperparameters['max_depth']:
              for min_samples_split in hyperparameters['min_samples_split']:
                  for min_samples_leaf in hyperparameters['min_samples_leaf']:
                      gbc = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, random_state=42)
                      gbc.fit(X_train, y_train)

                      # # Save the decision tree's rules to a text file
                      # tree_rules = export_text(gbc.estimators_[0], feature_names=X_train.columns.tolist())
                      # with open(ordner_link +"/"+f'decision_tree_rules_bt_{n_estimators}_{max_depth}_{min_samples_split}_{min_samples_leaf}_{H}_{test_size}_{testformat}.txt', 'w') as f:
                      #     f.write(tree_rules)
                      # Make predictions
                      y_pred = gbc.predict(X_test)

                      all_data = X_test.copy()
                      all_data['real_ID3'] = y_test
                      all_data['pred_ID3'] = y_pred

                      if max_depth is None:
                        max_depth_name = "None"
                      else:
                        max_depth_name = max_depth

                      write_dataframe_to_database(all_data, f'Table_{str(n_estimators)}_{str(max_depth_name)}_{str(min_samples_split)}_{str(min_samples_leaf)}_ypred', conn_bt_clf_, cur_bt_clf_, show_succ_msg = False)

                      # Calculate the metrics
                      accuracy = accuracy_score(y_test, y_pred)
                      classification_report_ = classification_report(y_test, y_pred)
                      confusion_matrix_ = confusion_matrix(y_test, y_pred)

                      new_row = {'n_estimators': n_estimators, 'learning_rate': learning_rate, 'max_depth': max_depth_name, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'accuracy': accuracy, 'classification_report': classification_report_, 'confusion_matrix': confusion_matrix_}
                      new_row_df = pd.DataFrame([new_row])
                      results = pd.concat([results, new_row_df], ignore_index=True)

  write_dataframe_to_database(results, f'results_error', conn_bt_clf_, cur_bt_clf_, show_succ_msg = False)

  conn_bt_clf_.close()
  return results